## Skipgram Word Vectors with Negative Sampling

Replicating Mikolov et al. 2013 with pytorch

Thanks to [this](https://github.com/DSKSD/DeepNLP-models-Pytorch) repo

In [2]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter

random.seed(1024)
torch.manual_seed(1)

In [32]:
flatten = lambda l: [item for sublist in l for item in sublist]

def get_batch(batch_size, train_data):
    random.shuffle(train_data)
    start_index = 0
    end_index = batch_size
    while end_index < len(train_data):
        batch = train_data[start_index:end_index]
        temp = end_index
        end_index = end_index + batch_size
        start_index = temp
        yield batch
    if end_index >= len(train_data):
        batch = train_data[start_index:]
        yield batch

def prepare_sequence(seq, word2idx):
    idxs = list(map(lambda w: word2idx[w] if word2idx.get(w) is not None else word2idx['<UNK>'], seq))
    return(autograd.Variable(torch.tensor(idxs, dtype = torch.long)))

def prepare_word(word, word2idx):
    return(autograd.Variable(torch.tensor([word2idx[word]], dtype = torch.long) if word2idx.get(word) is not None else torch.tensor(word2idx['<UNK>'], dtype = torch.long)))

In [8]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500]
corpus = [[word.lower() for word in sent] for sent in corpus]

word_count = Counter(flatten(corpus))

In [18]:
word_count.most_common()[:10]

[(',', 648),
 ('the', 533),
 ('.', 342),
 ('of', 299),
 ('a', 273),
 ('and', 258),
 ('in', 188),
 ('to', 182),
 ('--', 129),
 ('"', 120)]

In [11]:
MIN_COUNT = 3
exclude = []

for w, c in word_count.items():
    if c < MIN_COUNT:
        exclude.append(w)

vocab = list(set(flatten(corpus)) - set(exclude))
vocab.append('<UNK>')

In [15]:
word2idx = {'<UNK>': 0}
for word in vocab:
    if word2idx.get(word) is None:
        word2idx[word] = len(word2idx)

In [23]:
WINDOW_SIZE = 5
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])

windows[:10]

[('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']'),
 ('<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>'),
 ('<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>'),
 ('[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('<DUMMY>',
  '<DUMMY>',


In [24]:
# (Stores center word, context word) for skipgram with window = 5
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        # Dont include excluded word from vocab
        if window[i] in exclude or window[WINDOW_SIZE] in exclude:
            continue
        # Also dont include center word or <DUMMY> with the center word
        if i == WINDOW_SIZE or window[i] == '<DUMMY>':
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))

In [33]:
X_p = []
y_p = []

# Storing the (center, context) tuple in the form of indices
for pair in train_data:
    X_p.append(prepare_word(pair[0], word2idx).view(1, -1))
    y_p.append(prepare_word(pair[1], word2idx).view(1, -1))
    
train_data = list(zip(X_p, y_p))

### Negative Sampling

In [37]:
Z = 0.001
num_total_words = sum([c for w, c in word_count.items() if w not in exclude])
unigram_table = []

for word in vocab:
    unigram_table.extend([word] * int(((word_count[word]/num_total_words)**0.75)/Z))

unigram_table[:10]

['matter',
 'matter',
 'matter',
 'thought',
 'thought',
 'thought',
 'thought',
 'thought',
 'thought',
 'sub']

In [102]:
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        target_index = targets[i].data.tolist()[0]
        while len(nsample) < k:
            # sample from unigram (^ 0.75) distribution
            neg_word = random.choice(unigram_table)
            if word2idx[neg_word] == target_index:
                continue
            nsample.append(neg_word)
        neg_samples.append(prepare_sequence(nsample, word2idx).view(1, -1))
    return(torch.cat(neg_samples))

negative_sampling(torch.tensor([[174]]), unigram_table, 10)

tensor([[287, 281, 287, 320, 430, 106, 430, 320, 287, 378]])

In [103]:
# 174 = '('
idx2word = {v:k for k,v in word2idx.items()}

test = negative_sampling(torch.tensor([[174]]), unigram_table, 10)
print("Word at 174: '{}' \nNegative Samples: ".format(idx2word.get(174)))
for i in flatten(test.data.tolist()):
    print(idx2word.get(i))

Word at 174: '(' 
Negative Samples: 
,
letter
these
view
room
did
stern
or
to
sperm


## Defining the Model

In [105]:
class SGNS(nn.Module):
    
    def __init__(self, vocab_size, ndim):
        super(SGNS, self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, ndim) # Input Vectors
        self.embedding_u = nn.Embedding(vocab_size, ndim) # Output Vectors
        self.logsigmoid = nn.LogSigmoid()
    
        # Xavier Initialization
        initrange = (2.0/(vocab_size + ndim))**5
        self.embedding_v.weight.data.uniform_(-initrange, initrange)
        self.embedding_u.weight.data.uniform_(-0.0, 0.0)
     
    def forward(self, center_words, target_words, neg_words):
        center_vectors = self.embedding_v(center_words)
        target_vectors = self.embedding_u(target_words)
        neg_vectors = -self.embedding_u(neg_words)
        
        positive_score = target_vectors.bmm(center_vectors.transpose(1, 2)).squeeze(2)
        negative_score = torch.sum(neg_vectors.bmm(center_vectors.transpose(1, 2)).squeeze(2), 1).view(neg_vectors.size(0), -1)
        
        loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)
        
        return(-torch.mean(loss))
    
    def get_vectors(self, inputs):
        vectors = self.embedding_v(inputs)
        return(vectors)

## Training the Model

In [106]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100
NEG = 10

losses = []
model = SGNS(len(word2idx), EMBEDDING_SIZE)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [107]:
for epoch in range(EPOCH):
    for i, batch in enumerate(get_batch(BATCH_SIZE, train_data)):
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs)
        targets = torch.cat(targets)
        neg_samples = negative_sampling(targets, unigram_table, NEG)
        model.zero_grad()
        
        loss = model(inputs, targets, neg_samples)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.tolist())
    if epoch % 10 == 0:
        print("Epoch: %d, mean_loss = %0.2f" % (epoch, np.mean(losses)))
        losses = []

Epoch: 0, mean_loss = 1.04
Epoch: 10, mean_loss = 0.87
Epoch: 20, mean_loss = 0.85
Epoch: 30, mean_loss = 0.83
Epoch: 40, mean_loss = 0.81
Epoch: 50, mean_loss = 0.79
Epoch: 60, mean_loss = 0.78
Epoch: 70, mean_loss = 0.76
Epoch: 80, mean_loss = 0.74
Epoch: 90, mean_loss = 0.72


In [109]:
model.get_vectors(prepare_word('man', word2idx))

tensor([[ 0.1091, -0.2299,  0.0473,  0.5426,  0.0884, -0.1539, -0.0781,  0.1008,
         -0.1723,  0.1543, -0.3471,  0.3903,  0.1920, -0.8509, -0.0822, -0.9305,
         -0.2987,  0.0296, -0.1161, -0.6562, -0.6720,  0.0702,  0.2608,  0.2302,
         -0.6900, -0.2282, -0.2640,  0.2476,  0.5778,  0.0322]],
       grad_fn=<EmbeddingBackward>)

In [132]:
def nearest_neighbors(target, vocab, k = 10):
    target_vector = model.get_vectors(prepare_word(target, word2idx))
    
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target:
            continue
        vector = model.get_vectors(prepare_word(list(vocab)[i], word2idx))
        
        cosine_sim = F.cosine_similarity(target_vector, vector).data.tolist()
        similarities.append([vocab[i], cosine_sim[0]])
    return(sorted(similarities, key = lambda x: x[1], reverse = True)[:k])
        

In [137]:
nearest_neighbors("passenger", vocab, 10)

[['tell', 0.8739544153213501],
 ['passengers', 0.86533522605896],
 ['penny', 0.8481901288032532],
 ['am', 0.8349013328552246],
 ['ishmael', 0.8310784697532654],
 ['why', 0.8298293948173523],
 ['particular', 0.8054483532905579],
 ['jolly', 0.7964008450508118],
 ['again', 0.7944890260696411],
 ['thought', 0.7863433957099915]]